In [8]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, f1_score

In [9]:
def load_combined_dataset(json_path):
    """
    Reads the combined dataset JSON and returns it as a Python list of dicts.
    Each item has: { "tokens": [...], "ner_tags": [...], "intent": "..." }
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [10]:
def main():
    # -------------------------------------------------------
    # 1. Load & Prepare the Data
    # -------------------------------------------------------
    combined_dataset_path = "../train_test_split/train_combined.json"
    data = load_combined_dataset(combined_dataset_path)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    # df has columns: "tokens", "ner_tags", "intent"

    # Create a "text" column by joining tokens
    df["text"] = df["tokens"].apply(lambda ts: " ".join(ts))

    # Create a "entity_text" column by joining the NER tags
    df["entity_text"] = df["ner_tags"].apply(lambda tags: " ".join(tags))

    # The label
    y_intent = df["intent"]

    # -------------------------------------------------------
    # 2. Build a Feature Pipeline
    # -------------------------------------------------------
    text_tfidf = ("text_tfidf", TfidfVectorizer(), "text")
    entity_tfidf = ("entity_tfidf", TfidfVectorizer(), "entity_text")

    feature_union = ColumnTransformer(
        [text_tfidf, entity_tfidf],
        remainder="drop"
    )

    pipeline = Pipeline([
        ("features", feature_union),
        ("clf", LogisticRegression(max_iter=1000, random_state=42))
    ])
    
    # -------------------------------------------------------
    # 3. Train/Test Split & Fit
    # -------------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        df, y_intent, test_size=0.2, random_state=42
    )

    pipeline.fit(X_train, y_train)
    
    # -------------------------------------------------------
    # 4. Evaluate
    # -------------------------------------------------------
    y_pred = pipeline.predict(X_test)
    
    # Print the classification report (includes per-class F1)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Print the overall F1 score (macro average)
    macro_f1 = f1_score(y_test, y_pred, average="macro")
    print(f"Overall Macro F1 Score: {macro_f1:.3f}")
    
    # (Optional) You could also print other averages:
    # weighted_f1 = f1_score(y_test, y_pred, average="weighted")
    # print(f"Overall Weighted F1 Score: {weighted_f1:.3f}")

In [11]:
if __name__ == "__main__":
    main()

Classification Report:
                           precision    recall  f1-score   support

             abbreviation       0.82      0.90      0.86        20
                 aircraft       0.91      0.67      0.77        15
aircraft+flight+flight_no       0.00      0.00      0.00         1
                  airfare       0.89      0.87      0.88        86
                  airline       0.97      0.76      0.85        37
                  airport       1.00      0.33      0.50         3
                 capacity       1.00      1.00      1.00         2
                     city       0.00      0.00      0.00         6
                 distance       1.00      1.00      1.00         1
                   flight       0.95      0.99      0.97       752
           flight+airfare       0.00      0.00      0.00         5
                flight_no       0.00      0.00      0.00         4
              flight_time       0.67      0.75      0.71         8
              ground_fare       0.00  

C:\Users\kanat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\kanat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\kanat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classificati